In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
tem_df=pd.read_csv('IMDB Dataset.csv')

In [3]:
tem_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
tem_df.duplicated().sum()

418

In [6]:
tem_df.drop_duplicates(inplace=True)

In [7]:
import re
def remove_tags(x):
    cleaned_text=re.sub(re.compile('<.*?>'),'',x)
    return cleaned_text


In [8]:
tem_df['review']=tem_df['review'].apply(remove_tags)

In [9]:
tem_df['review']=tem_df['review'].apply(lambda x:x.lower())

In [10]:
from nltk.corpus import stopwords
sw_list=stopwords.words('english')
tem_df['review']=tem_df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [12]:
import gensim
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize

In [13]:
story=[]
for doc in tem_df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [14]:
model=gensim.models.Word2Vec(window=10,min_count=2)

In [16]:
model.build_vocab(story)

In [17]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(29540101, 30891345)

In [18]:
len(model.wv.index_to_key)

61843

In [19]:
def document(doc):
    #remove out of vocabulary words
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [21]:
document(tem_df['review'].values[0])

array([ 0.0040476 ,  0.3671775 ,  0.16372456, -0.11327665,  0.4337677 ,
       -0.3908572 , -0.11353175,  0.51358724,  0.34204268, -0.07390527,
        0.2864596 , -0.49708745,  0.38403988,  0.21840471, -0.05182619,
       -0.11554765, -0.3129528 ,  0.11764617,  0.1128199 , -0.29158717,
       -0.02593501,  0.14656602,  0.12091761,  0.10006158, -0.4176713 ,
        0.01728558,  0.18331121,  0.07271238, -0.37685066, -0.41313067,
        0.41407436,  0.14626533,  0.1253638 , -0.17696963, -0.23097716,
        0.15838438, -0.0573775 , -0.21685043,  0.07592293, -0.33772695,
        0.29515535, -0.23596753, -0.294794  , -0.412719  ,  0.03931681,
       -0.12397474, -0.22806522,  0.00359492,  0.01576542,  0.19710214,
        0.31236798, -0.14864914, -0.19133632, -0.61817694, -0.12563083,
        0.27749112,  0.40750533, -0.44873896, -0.11579973,  0.23894909,
        0.08677965,  0.14739074, -0.41777474, -0.09356087, -0.29719782,
       -0.06351691, -0.32662496, -0.327905  , -0.00477651, -0.45

In [22]:
from tqdm import tqdm

In [25]:
#x=[]
#for doc in tqdm(tem_df['review'].values):
    #x.append(document(doc))